In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
import obspy
import os
from obspy.clients.fdsn import Client
from obspy.clients.syngine import Client as SyngineClient
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
from obspy.io.xseed import Parser
from obspy.signal.cross_correlation import xcorr,xcorr_3c

# Coletando os dados das estações BP

In [3]:
ev_list = []
ev_listS = []

for root, dirs, files in os.walk('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/Mantle_Transition_Migration_NO_PP/MTZ_data/0.5/'):
    for datafile in files:
        if datafile.endswith('.eqr'):
            ev_list.append(os.path.join(root, datafile))
ev_listS = sorted(ev_list)

In [4]:
ev = obspy.Stream()
for i,j in enumerate(ev_listS):
    ev += obspy.read(j,headonly=True)

In [8]:
event_DD = []
event_MM = []
event_YYYY = []
event_hh = []
event_mm = []
event_julday = []
event_depth = []
event_lat = []
event_long = []
event_dist = []
event_gcarc = []
event_sta = []
event_channel = []
sta_lat = []
sta_long = []
sta_channel = []
event_starttime = []
event_endtime = []


for i,j in enumerate(ev):
    #Calculando o parâmetro do raio
    model = TauPyModel(model="iasp91")
    arrivals = model.get_travel_times(source_depth_in_km=j.stats.sac.evdp, distance_in_degree=j.stats.sac.gcarc, phase_list=["P"])
    arr = arrivals[0]
    
    event_time = obspy.UTCDateTime(year=j.stats.sac.nzyear, julday=j.stats.sac.nzjday)
    event_DD.append("{0:02.0f}".format(event_time.day))
    event_MM.append("{0:02.0f}".format(event_time.month))
    event_YYYY.append(j.stats.sac.nzyear)
    event_hh.append("{0:02.0f}".format(j.stats.sac.nzhour))
    event_mm.append("{0:02.0f}".format(j.stats.sac.nzmin-arr.time/60))
    event_julday.append(j.stats.sac.nzjday)
    event_depth.append(j.stats.sac.evdp)
    event_lat.append(j.stats.sac.evla)
    event_long.append(j.stats.sac.evlo)
    event_dist.append(j.stats.sac.dist)
    event_gcarc.append(j.stats.sac.gcarc)
    event_sta.append(j.stats.station)
    sta_lat.append(j.stats.sac.stla)
    sta_long.append(j.stats.sac.stlo)

In [11]:
eventid_lst = []
for k,l in enumerate(ev):
    eventid_lst.append('GCMT:C'+str(event_YYYY[k])+str(event_MM[k])+str(event_DD[k])+str(event_hh[k])+str(event_mm[k])+'A')

In [12]:
eventid_lst

['GCMT:C2015021119-3A',
 'GCMT:C2015021319-1A',
 'GCMT:C201502162200A',
 'GCMT:C201503181827A',
 'GCMT:C201506171251A',
 'GCMT:C201506200209A',
 'GCMT:C201512171949A',
 'GCMT:C2015021319-1A',
 'GCMT:C201503181827A',
 'GCMT:C201506171251A',
 'GCMT:C201506200209A',
 'GCMT:C201512171949A',
 'GCMT:C201512191925A',
 'GCMT:C201604151410A',
 'GCMT:C201604200833A',
 'GCMT:C201605280946A',
 'GCMT:C201608190731A',
 'GCMT:C201608191733A',
 'GCMT:C201610272032A',
 'GCMT:C201610300640A',
 'GCMT:C2016110805-5A',
 'GCMT:C201611241843A',
 'GCMT:C201612181329A',
 'GCMT:C201612251421A',
 'GCMT:C201701060630A',
 'GCMT:C201501070507A',
 'GCMT:C2015021319-1A',
 'GCMT:C201502162200A',
 'GCMT:C201506171251A',
 'GCMT:C201506200209A',
 'GCMT:C201507290009A',
 'GCMT:C201512171949A',
 'GCMT:C201602100033A',
 'GCMT:C201604200833A',
 'GCMT:C201604230124A',
 'GCMT:C201604250706A',
 'GCMT:C201604290133A',
 'GCMT:C201607251726A',
 'GCMT:C201609101008A',
 'GCMT:C201611241843A',
 'GCMT:C201612181329A',
 'GCMT:C20160121

In [13]:
#initial_time = [str(i) for i in event_starttime]
#initial_time = sorted(list(set(initial_time)))
#starttime_lst = [obspy.UTCDateTime(i) for i in initial_time] 

In [14]:
lat_lst = sta_lat
long_lst = sta_long
sta_lst = event_sta

## Calculando os sismogramas sintéticos através do servidor da Syngine IRIS

In [15]:
%%time

c_s = SyngineClient()
#model = ["ak135f_2s", "iasp91_2s", "prem_a_2s","prem_i_2s"]
model = "iasp91_2s"


st_synth = []
for k,l in enumerate(eventid_lst):
    try:
        print('Evento '+l)
        st_synth.append(obspy.Stream(c_s.get_waveforms(model=model, 
                                                receiverlatitude=lat_lst[k],
                                                receiverlongitude=long_lst[k],
                                                networkcode='PB',
                                                stationcode=sta_lst[k],
                                                eventid=eventid_lst[k],
                                                dt="0.01",units="displacement",starttime="P-10", endtime="P+260",
                                                format='saczip')))
    except:
        print('Problema no Evento: '+eventid_lst[k])

Evento GCMT:C2015021119-3A
Problema no Evento: GCMT:C2015021119-3A
Evento GCMT:C2015021319-1A
Problema no Evento: GCMT:C2015021319-1A
Evento GCMT:C201502162200A
Evento GCMT:C201503181827A
Evento GCMT:C201506171251A
Evento GCMT:C201506200209A
Problema no Evento: GCMT:C201506200209A
Evento GCMT:C201512171949A
Evento GCMT:C2015021319-1A
Problema no Evento: GCMT:C2015021319-1A
Evento GCMT:C201503181827A
Evento GCMT:C201506171251A
Evento GCMT:C201506200209A
Problema no Evento: GCMT:C201506200209A
Evento GCMT:C201512171949A
Evento GCMT:C201512191925A
Evento GCMT:C201604151410A
Problema no Evento: GCMT:C201604151410A
Evento GCMT:C201604200833A
Evento GCMT:C201605280946A
Problema no Evento: GCMT:C201605280946A
Evento GCMT:C201608190731A
Problema no Evento: GCMT:C201608190731A
Evento GCMT:C201608191733A
Evento GCMT:C201610272032A
Evento GCMT:C201610300640A
Evento GCMT:C2016110805-5A
Problema no Evento: GCMT:C2016110805-5A
Evento GCMT:C201611241843A
Evento GCMT:C201612181329A
Problema no Evento:

Problema no Evento: GCMT:C201706140728A
Evento GCMT:C201706221230A
Problema no Evento: GCMT:C201706221230A
Evento GCMT:C201707202231A
Evento GCMT:C201709080449A
Evento GCMT:C201709191814A
Evento GCMT:C2017092313-7A
Problema no Evento: GCMT:C2017092313-7A
Evento GCMT:C2017101019-8A
Problema no Evento: GCMT:C2017101019-8A
Evento GCMT:C201711101201A
Evento GCMT:C201711110036A
Evento GCMT:C201711130228A
Evento GCMT:C201801100251A
Evento GCMT:C201801140918A
Evento GCMT:C201802162339A
Evento GCMT:C201804101019A
Evento GCMT:C201604151411A
Evento GCMT:C201604220303A
Evento GCMT:C201604290133A
Evento GCMT:C201605181646A
Evento GCMT:C201605280946A
Problema no Evento: GCMT:C201605280946A
Evento GCMT:C201607251726A
Evento GCMT:C201608190732A
Evento GCMT:C201608210345A
Evento GCMT:C201610272032A
Evento GCMT:C201610300640A
Evento GCMT:C2016110805-5A
Problema no Evento: GCMT:C2016110805-5A
Evento GCMT:C2016112021-2A
Problema no Evento: GCMT:C2016112021-2A
Evento GCMT:C201611241844A
Problema no Evento

Evento GCMT:C201608190732A
Evento GCMT:C201608210345A
Evento GCMT:C201608240136A
Evento GCMT:C2016110805-5A
Problema no Evento: GCMT:C2016110805-5A
Evento GCMT:C2016112021-3A
Problema no Evento: GCMT:C2016112021-3A
Evento GCMT:C201611241843A
Evento GCMT:C201612251421A
Problema no Evento: GCMT:C201612251421A
Evento GCMT:C201706140729A
Evento GCMT:C201706221231A
Evento GCMT:C201706302230A
Problema no Evento: GCMT:C201706302230A
Evento GCMT:C201707202230A
Problema no Evento: GCMT:C201707202230A
Evento GCMT:C201709080449A
Evento GCMT:C201709191814A
Evento GCMT:C2017092313-8A
Problema no Evento: GCMT:C2017092313-8A
Evento GCMT:C201709241005A
Problema no Evento: GCMT:C201709241005A
Evento GCMT:C2017101019-8A
Problema no Evento: GCMT:C2017101019-8A
Evento GCMT:C201711101201A
Evento GCMT:C201801100250A
Problema no Evento: GCMT:C201801100250A
Evento GCMT:C201801281602A
Problema no Evento: GCMT:C201801281602A
Evento GCMT:C201604190525A
Evento GCMT:C201604220303A
Evento GCMT:C201604290133A
Evento

Problema no Evento: GCMT:C2014020104-2A
Evento GCMT:C201403110243A
Problema no Evento: GCMT:C201403110243A
Evento GCMT:C201403152350A
Problema no Evento: GCMT:C201403152350A
Evento GCMT:C201403171325A
Problema no Evento: GCMT:C201403171325A
Evento GCMT:C2014041504-3A
Problema no Evento: GCMT:C2014041504-3A
Evento GCMT:C201404181427A
Evento GCMT:C201405081700A
Evento GCMT:C2014062908-8A
Problema no Evento: GCMT:C2014062908-8A
Evento GCMT:C201408232232A
Evento GCMT:C2014090607-8A
Problema no Evento: GCMT:C2014090607-8A
Evento GCMT:C201410090214A
Evento GCMT:C201410100408A
Problema no Evento: GCMT:C201410100408A
Evento GCMT:C201410140351A
Evento GCMT:C201412071211A
Evento GCMT:C201412072116A
Evento GCMT:C2015021319-1A
Problema no Evento: GCMT:C2015021319-1A
Evento GCMT:C201502162200A
Evento GCMT:C201502221422A
Problema no Evento: GCMT:C201502221422A
Evento GCMT:C201505191525A
Evento GCMT:C201506171251A
Evento GCMT:C201506200209A
Problema no Evento: GCMT:C201506200209A
Evento GCMT:C2016021

In [17]:
loc_folder = '/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/Mantle_Transition_Migration_NO_PP_SYN/MTZ_data/'

In [18]:
%%time

for k,l in enumerate(st_synth):
    for r,t in enumerate(l):
        evdp = t.stats.sac.evdp/1000
        gcarc = t.stats.sac.gcarc
            #Calculando o parâmetro do raio
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=evdp, distance_in_degree=gcarc, phase_list=["P"])
        arr = arrivals[0]
        j = t.stats.starttime
        t.stats.sac.user8 = arr.ray_param/6371
        folder_loc_string = loc_folder+t.stats.station+'/'+str(j.year)+'/'+str("{0:0=3d}".format(j.julday))+'/'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)
        os.makedirs(folder_loc_string,exist_ok=True)
        t.write(folder_loc_string+'/SYN.'+t.stats.network+'.'+t.stats.station+'.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+t.stats.channel[-1],format='SAC')

CPU times: user 1min 2s, sys: 433 ms, total: 1min 3s
Wall time: 1min 4s
